<a href="https://colab.research.google.com/github/wsno1-lim/DF-AI2team/blob/main/99.%EC%8A%A4%ED%84%B0%EB%94%94/Kaggle/%5BKaggle_1%5D_Stock_Market_(Technical_Indicators)_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주가 분석 기술 지표 구현
---
https://www.kaggle.com/tomfarm/stock-market-technical-indicators-visualization/edit  
* Huge Stock Market Dataset 을 기반으로 함
* 요약 : 주가 분석 기술 지표 여러가지를 구현해놓았다. 또한 각 지표에 대해서 시각화 할 수 있는 함수들도 구현해 놓았음! 나중에 가져다가 사용하면 됨.
* 이 사람이 작성한 다른 문서(LSTM 예측)
  - 위에서 작성한 여러 지표들로 예측하기  
https://www.kaggle.com/kratisaxena/lstm-gru-models-for-stock-movement-analysis

In [3]:
import numpy as np
import pandas as pd
import os
import random
import copy
import matplotlib.pyplot as plt

In [4]:
# 서브프로세스 실행 후 출력문자열 리턴
from subprocess import check_output

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Stocks 폴더 안에 있는 파일 개수
os.chdir('/content/drive/MyDrive/hsmd/')
list = os.listdir()
number_files = len(list)
print(number_files)

10


In [7]:
# 데이터 중 8개를 랜덤으로 뽑아 분석해본다
filenames = random.sample([x for x in list if x.endswith('.txt') and os.path.getsize(os.path.join('', x)) > 0], 8)
print(filenames)

['ncb.us.txt', 'abac.us.txt', 'nby.us.txt', 'aau.us.txt', 'nan.us.txt', 'naov.us.txt', 'nblx.us.txt', 'cern.us.txt']


In [8]:
# 뽑은 데이터를 데이터프레임으로 만들고, df의 리스트를 생성한다
data = []
for fn in filenames :
    df = pd.read_csv(os.path.join('', fn), sep=',')
    label, _, _ = fn.split(sep='.')
    df['Label'] = label
    df['Date'] = pd.to_datetime(df['Date'])
    data.append(df)

In [9]:
data[0].head()

,Date,Open,High,Low,Close,Volume,OpenInt,Label
0,2009-04-27,10.477,10.477,10.477,10.477,0,0,ncb
1,2009-04-28,10.477,10.477,10.208,10.477,168518,0,ncb
2,2009-04-29,10.477,10.484,10.477,10.477,40489,0,ncb
3,2009-04-30,10.477,10.477,10.477,10.477,14862,0,ncb
4,2009-05-01,10.477,10.477,10.439,10.439,1277,0,ncb


## 데이터 프레임 안에 다양한 기술 지표 추가하기  
---
여기서 다룰 기술 지표들은 다음과 같습니다.
* RSI, Volume (plain), Bollinger Bands, Aroon, PVT, acceleration bands
* Stochastic, Chaikin Money Flow, Parabolic SAR, Rate of Change, Volume weighted average Price, momentum
* Commodity Channel Index, On Balance Volume, Keltner Channels, Triple Exponential Moving Average, Normalized Averager True Range, directional movement indicators
* MACD, Money flowindex, Ichimoku, William %R, Volume MINMAX, adaptive moving average  
  
아래는 기본적인 몇가지만 복사해놓았고, 나머지는 링크에서 가져다 사용하면 됨

In [10]:
TechIndicator = copy.deepcopy(data)
TechIndicator

[           Date    Open    High     Low   Close  Volume  OpenInt Label
 0    2009-04-27  10.477  10.477  10.477  10.477       0        0   ncb
 1    2009-04-28  10.477  10.477  10.208  10.477  168518        0   ncb
 2    2009-04-29  10.477  10.484  10.477  10.477   40489        0   ncb
 3    2009-04-30  10.477  10.477  10.477  10.477   14862        0   ncb
 4    2009-05-01  10.477  10.477  10.439  10.439    1277        0   ncb
 ...         ...     ...     ...     ...     ...     ...      ...   ...
 2125 2017-11-06  17.140  17.140  16.790  16.900   12289        0   ncb
 2126 2017-11-07  17.299  17.299  17.000  17.050    7388        0   ncb
 2127 2017-11-08  17.076  17.170  17.076  17.150    1212        0   ncb
 2128 2017-11-09  17.190  17.220  17.120  17.150    2596        0   ncb
 2129 2017-11-10  17.100  17.100  16.950  16.950   16812        0   ncb
 
 [2130 rows x 8 columns],
            Date     Open     High      Low    Close  Volume  OpenInt Label
 0    2010-07-19  24.0000  24.00

* RSI 계산하기

In [11]:
# RSI (상대강도지수) : 주가의 상승압력과 하락압력간의 상대적 강도 
# RSI 가 클수록 주가의 상승추세가 크고, 작을수록 주가의 하락추세가 크다
# up/down - N일동안의 상승폭/하락폭 평균, N은 일반적으로 14일
# RSI 시그널 : N`일 이동평균선, N`는 일반적으로 6일
# RSI 70% 이상, 시그널 하향돌파(데드크로스) 시 매도 / 30% 이하, 시그널 상향돌파(골든크로스) 시 매수
def rsi(values) :
    up = values[values > 0].mean()
    down = -1 * values[values < 0].mean()
    return 100 * up / (up + down)

In [12]:
# TechIndicator 안에 있는 8개 종목에 대해 각각 RSI 구하기
for stock in range(len(TechIndicator)) :
    TechIndicator[stock]['Momentum_1D'] = (TechIndicator[stock]['Close'] - TechIndicator[stock]['Close'].shift(1)).fillna(0)
    TechIndicator[stock]['RSI_14D'] = TechIndicator[stock]['Momentum_1D'].rolling(center=False, window=14).apply(rsi).fillna(0)
TechIndicator[0].tail(5)

,Date,Open,High,Low,Close,Volume,OpenInt,Label,Momentum_1D,RSI_14D
2125,2017-11-06,17.140,17.140,16.790,16.90,12289,0,ncb,-0.10,30.583501
2126,2017-11-07,17.299,17.299,17.000,17.05,7388,0,ncb,0.15,29.447853
2127,2017-11-08,17.076,17.170,17.076,17.15,1212,0,ncb,0.10,31.137725
2128,2017-11-09,17.190,17.220,17.120,17.15,2596,0,ncb,0.00,30.434783
2129,2017-11-10,17.100,17.100,16.950,16.95,16812,0,ncb,-0.20,28.526646


* 거래량 : 기존 거래량 칼럼에 있는 값을 사용한다

In [13]:
for stock in range(len(TechIndicator)) :
    TechIndicator[stock]['Volume_plain'] = TechIndicator[stock]['Volume'].fillna(0)
TechIndicator[0].tail()

,Date,Open,High,Low,Close,Volume,OpenInt,Label,Momentum_1D,RSI_14D,Volume_plain
2125,2017-11-06,17.140,17.140,16.790,16.90,12289,0,ncb,-0.10,30.583501,12289
2126,2017-11-07,17.299,17.299,17.000,17.05,7388,0,ncb,0.15,29.447853,7388
2127,2017-11-08,17.076,17.170,17.076,17.15,1212,0,ncb,0.10,31.137725,1212
2128,2017-11-09,17.190,17.220,17.120,17.15,2596,0,ncb,0.00,30.434783,2596
2129,2017-11-10,17.100,17.100,16.950,16.95,16812,0,ncb,-0.20,28.526646,16812


* 볼린저 밴드 계산하기

In [14]:
def bbands(price, length=30, numsd=2) :
    ave = price.rolling(window=length, center=False).mean()
    sd = price.rolling(window=length, center=False).std()
    upb = ave + (sd * numsd)
    dnb = ave - (sd * numsd)
    return np.round(ave, 3), np.round(upb, 3), np.round(dnb, 3)

In [15]:
for stock in range(len(TechIndicator)) :
    TechIndicator[stock]['BB_Middle_Band'], TechIndicator[stock]['BB_Upper_Band'], TechIndicator[stock]['BB_Lower_Band'] = bbands(TechIndicator[stock]['Close'], length=20, numsd=1)
    TechIndicator[stock]['BB_Middle_Band'].fillna(0)
    TechIndicator[stock]['BB_Upper_Band'].fillna(0)
    TechIndicator[stock]['BB_Lower_Band'].fillna(0)
TechIndicator[0].tail()

,Date,Open,High,Low,Close,Volume,OpenInt,Label,Momentum_1D,RSI_14D,Volume_plain,BB_Middle_Band,BB_Upper_Band,BB_Lower_Band
2125,2017-11-06,17.140,17.140,16.790,16.90,12289,0,ncb,-0.10,30.583501,12289,17.380,17.768,16.992
2126,2017-11-07,17.299,17.299,17.000,17.05,7388,0,ncb,0.15,29.447853,7388,17.362,17.757,16.967
2127,2017-11-08,17.076,17.170,17.076,17.15,1212,0,ncb,0.10,31.137725,1212,17.347,17.744,16.950
2128,2017-11-09,17.190,17.220,17.120,17.15,2596,0,ncb,0.00,30.434783,2596,17.318,17.705,16.930
2129,2017-11-10,17.100,17.100,16.950,16.95,16812,0,ncb,-0.20,28.526646,16812,17.283,17.670,16.895


* Aroon Oscillator 계산

In [16]:
def aroon(df, tf=25):
    aroonup = []
    aroondown = []
    x = tf
    while x< len(df['Date']):
        aroon_up = ((df['High'][x-tf:x].tolist().index(max(df['High'][x-tf:x])))/float(tf))*100
        aroon_down = ((df['Low'][x-tf:x].tolist().index(min(df['Low'][x-tf:x])))/float(tf))*100
        aroonup.append(aroon_up)
        aroondown.append(aroon_down)
        x+=1
    return aroonup, aroondown

In [17]:
for stock in range(len(TechIndicator)):
    listofzeros = [0] * 25
    up, down = aroon(TechIndicator[stock])
    aroon_list = [x - y for x, y in zip(up,down)]
    if len(aroon_list)==0:
        aroon_list = [0] * TechIndicator[stock].shape[0]
        TechIndicator[stock]['Aroon_Oscillator'] = aroon_list
    else:
        TechIndicator[stock]['Aroon_Oscillator'] = listofzeros+aroon_list

* Price Volume Trend 계산  
PVT = [((CurrentClose - PreviousClose) / PreviousClose) x Volume] + PreviousPVT

In [18]:
for stock in range(len(TechIndicator)):
    TechIndicator[stock]["PVT"] = (TechIndicator[stock]['Momentum_1D']/ TechIndicator[stock]['Close'].shift(1))*TechIndicator[stock]['Volume']
    TechIndicator[stock]["PVT"] = TechIndicator[stock]["PVT"]-TechIndicator[stock]["PVT"].shift(1)
    TechIndicator[stock]["PVT"] = TechIndicator[stock]["PVT"].fillna(0)
TechIndicator[0].tail()

,Date,Open,High,Low,Close,Volume,OpenInt,Label,Momentum_1D,RSI_14D,Volume_plain,BB_Middle_Band,BB_Upper_Band,BB_Lower_Band,Aroon_Oscillator,PVT
2125,2017-11-06,17.140,17.140,16.790,16.90,12289,0,ncb,-0.10,30.583501,12289,17.380,17.768,16.992,-48.0,-98.696519
2126,2017-11-07,17.299,17.299,17.000,17.05,7388,0,ncb,0.15,29.447853,7388,17.362,17.757,16.967,-52.0,137.862200
2127,2017-11-08,17.076,17.170,17.076,17.15,1212,0,ncb,0.10,31.137725,1212,17.347,17.744,16.950,-52.0,-58.465460
2128,2017-11-09,17.190,17.220,17.120,17.15,2596,0,ncb,0.00,30.434783,2596,17.318,17.705,16.930,-52.0,-7.108504
2129,2017-11-10,17.100,17.100,16.950,16.95,16812,0,ncb,-0.20,28.526646,16812,17.283,17.670,16.895,-52.0,-196.058309


* Acceleration Bands 계산

In [19]:
def abands(df):
    #df['AB_Middle_Band'] = pd.rolling_mean(df['Close'], 20)
    df['AB_Middle_Band'] = df['Close'].rolling(window = 20, center=False).mean()
    # High * ( 1 + 4 * (High - Low) / (High + Low))
    df['aupband'] = df['High'] * (1 + 4 * (df['High']-df['Low'])/(df['High']+df['Low']))
    df['AB_Upper_Band'] = df['aupband'].rolling(window=20, center=False).mean()
    # Low *(1 - 4 * (High - Low)/ (High + Low))
    df['adownband'] = df['Low'] * (1 - 4 * (df['High']-df['Low'])/(df['High']+df['Low']))
    df['AB_Lower_Band'] = df['adownband'].rolling(window=20, center=False).mean()

In [20]:
for stock in range(len(TechIndicator)):
    abands(TechIndicator[stock])
    TechIndicator[stock] = TechIndicator[stock].fillna(0)
TechIndicator[0].tail()

,Date,Open,High,Low,Close,Volume,OpenInt,Label,Momentum_1D,RSI_14D,Volume_plain,BB_Middle_Band,BB_Upper_Band,BB_Lower_Band,Aroon_Oscillator,PVT,AB_Middle_Band,aupband,AB_Upper_Band,adownband,AB_Lower_Band
2125,2017-11-06,17.140,17.140,16.790,16.90,12289,0,ncb,-0.10,30.583501,12289,17.380,17.768,16.992,-48.0,-98.696519,17.38010,17.847221,18.044471,16.097221,16.794821
2126,2017-11-07,17.299,17.299,17.000,17.05,7388,0,ncb,0.15,29.447853,7388,17.362,17.757,16.967,-52.0,137.862200,17.36205,17.902213,18.037880,16.407213,16.770980
2127,2017-11-08,17.076,17.170,17.076,17.15,1212,0,ncb,0.10,31.137725,1212,17.347,17.744,16.950,-52.0,-58.465460,17.34700,17.358516,17.996683,16.888516,16.776033
2128,2017-11-09,17.190,17.220,17.120,17.15,2596,0,ncb,0.00,30.434783,2596,17.318,17.705,16.930,-52.0,-7.108504,17.31750,17.420582,17.928160,16.920582,16.802260
2129,2017-11-10,17.100,17.100,16.950,16.95,16812,0,ncb,-0.20,28.526646,16812,17.283,17.670,16.895,-52.0,-196.058309,17.28250,17.401322,17.909721,16.651322,16.756321


> 기타 다른 지표에 대해서는 링크에서 직접 참조